In [180]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import scipy.stats as stats
from scipy.stats import ttest_rel

# create a function for performing the paired t-test 
#     and for calculating the effect size of the paired t-test
#     and also returning the means of the two samples 
def paired_ttest(x,y,alternative='two-sided'):
    x_arr = np.array(x)
    y_arr = np.array(y)
    # t-test from scipy.stats
    t_statistic, p_value = ttest_rel(x_arr,y_arr,alternative=alternative)
    # Calculate effect size (Cohen's d)
    # calculate mean of differences between two arrays
    mean_diff = np.mean(x_arr - y_arr)
    mean_x = np.mean(x_arr)
    mean_y = np.mean(y_arr)
    pooled_std = np.sqrt((np.std(x_arr) ** 2 + np.std(y_arr) ** 2) / 2)
    d = mean_diff / pooled_std
    return t_statistic, p_value, d, mean_x, mean_y

# create a function for performing the shapiro-wilk test for normality
#     of differences between two paired samples and for creating a qq plot
#     of the differences between the two paired samples
def shapiro_wilk_qq(x,y,recon1,recon2,walk_length):
    # shapiro-wilk test for normality
    shapiro_stat, shapiro_p = stats.shapiro(x-y)
    # # make figure
    # plt.figure(figsize=(8,6))
    # # qq plot
    # qq = stats.probplot(x-y, dist="norm", plot=plt)
    # # plot name
    # # qq_plot_name = '/datain/dataset/plots/normality/qq_plot_differences'+str(x)+'_'+str(y)+'.png' # Uncomment for use in singularity
    # qq_plot_name = '/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/plots/normality/qq_plot_differences'+str(recon1)+'_'+str(recon2)+'_walklength_'+str(walk_length)+'.png'
    # # make matplotlib plot
    # plt.title('QQ Plot for Differences between '+str(recon1)+' and '+str(recon2))
    # plt.xlabel('Theoretical Quantiles')
    # plt.ylabel('Ordered Values')
    # plt.tight_layout()
    # # save qq plot
    # plt.savefig(qq_plot_name)
    # plt.close()
    return shapiro_stat, shapiro_p

# create a function for performing the wilcoxon signed rank test with paired samples 
#     and calculating the effect size of the paired samples wilcoxon signed rank test
#     and also returning the medians of the two samples
def wilcoxon_test(x,y,alternative='two-sided'):
    x_arr = np.array(x)
    y_arr = np.array(y)
    w_statistic, p_value = stats.wilcoxon(x_arr,y_arr,alternative=alternative,zero_method='pratt',mode='exact')
    median_x = np.median(x_arr)
    median_y = np.median(y_arr)
    # Calculate effect size (r)
    r = w_statistic / (len(x_arr) * (len(y_arr) + 1) / 2)
    return w_statistic, p_value, r, median_x, median_y


# create a function that reads a dataframe and returns a dataframe with the
#     values of column i for which the row values in the 'Recon 1' column equal to recon1 
#     and the values in the 'Recon 2' column equal to recon2
def get_df_int(df,recon1,recon2,i):
    if i is not None:
        df_i = df[(df['Recon 1'][i] == recon1) & (df['Recon 2'][i] == recon2)]
    else:
        df_i = df[(df['Recon 1'] == recon1) & (df['Recon 2'] == recon2)]
    return df_i

# # create a function for calculating the mean, standard deviation, and median of the
# #     values in each column of a dataframe - where the 'Recon 1' is equal to recon1
# #     and 'Recon 2' is equal to recon2
# def get_stats(df,recon1,recon2):
#     df_i = df.loc[(df['Recon 1'] == recon1) & (df['Recon 2'] == recon2)]
#     mean = df_i.mean(axis=0)
#     std = df_i.std(axis=0)
#     median = df_i.median(axis=0)
#     return mean, std, median

def print_stats(df):
    print('Mean:')
    print(df.mean(axis=0))
    print('Standard Deviation:')
    print(df.std(axis=0))
    print('Median:')
    print(df.median(axis=0))
    print('Min:')
    print(df.min(axis=0))
    print('Max:')
    print(df.max(axis=0))


Read in files and perform statistical tests of independence for Pearson scores at each walk length

In [214]:
# Read data
main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/'
verbose = False
# main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v3/scrambled_dataset/'
file_list = Path(main_path).glob('*batch?.csv')
for file in file_list:
    if verbose is True:
        print(file)
    df = pd.read_csv(file)
    # ,ave to a copy of df, then perform Fisher's r to z transformation on Pearson scores
    df_z = df.copy()
    for i in range(1, 6):
        df_z[str(i)] = np.arctanh(df_z[str(i)])
    # print(df_z)
    if verbose is True:
        print(file.stem)
    # compare difference in score between Recon methods for each walk length column
    for i in range(1, 6):
        if verbose is True:
            print('Walk Length: '+str(i))
        # get unique values of Recon column
        recon_list = df['Recon'].unique()
        df_stats_all = pd.DataFrame()
        df_stats = pd.DataFrame()
        # loop through each unique value of Recon column
        for recon in recon_list:
            if verbose is True:
                print(recon)
            # combine statistics from each Recon method into dataframe
            df_stats = pd.DataFrame({'Recon': recon_list, 'Mean': df[df['Recon'] == str(recon)][str(i)].mean(), 'Stdev': df[df['Recon'] == str(recon)][str(i)].std(
            ), 'Median': df[df['Recon'] == str(recon)][str(i)].median(), 'IQR': df[df['Recon'] == str(recon)][str(i)].quantile(q=0.75)-df[df['Recon'] == str(recon)][str(i)].quantile(q=0.25)})
            # combine df_stats from each Recon method into dataframe
            df_stats_all = pd.concat([df_stats, df_stats_all], axis=0)
        # print(df_stats_all)
        # save dataframe to csv
        df_stats_all.to_csv(main_path+'/stats/'+file.stem +
                            '_walk_length_'+str(i)+'_stats.csv', index=False)
        # calculate p-value for difference in score between Recon methods in recon_list, with comparisons between each pair of Recon methods, save all statistics and p-values to csv
        df_z_ttest_rel_results = pd.DataFrame()
        df_z_shapiro_wilk_results = pd.DataFrame()
        df_z_wilcoxon_results = pd.DataFrame()
        for recon in recon_list:
            for recon2 in recon_list:
                if recon != recon2:
                    if verbose is True:
                        print(recon)
                        print(recon2)
                    # calculate t-test on z-scores, get t-statistic and p-value and effect size, means
                    ttest_rel_z_score = paired_ttest(df_z[df_z['Recon'] == str(
                        recon)][str(i)], df_z[df_z['Recon'] == str(recon2)][str(i)], alternative='less')
                    if verbose:
                        print(ttest_rel_z_score[2])
                    # calculate shapiro-wilk test on z-scores, get W-statistic and p-value
                    # get array of df_z values for recon and recon2
                    recon_1_z_scores = np.array(df_z[df_z['Recon'] == str(recon)][str(i)])
                    recon_2_z_scores = np.array(df_z[df_z['Recon'] == str(recon2)][str(i)])
                    # calculate differences between arrays of recon and recon2 z-scores
                    differences = recon_1_z_scores - recon_2_z_scores
                    shapiro_wilk_w, shapiro_wilk_p = stats.shapiro(differences)
                    if verbose:
                        print(shapiro_wilk_p)
                    # calculate wilcoxon test on z-scores, get test statistic and p-value and effect size, medians
                    wilcoxon_results = wilcoxon_test(df_z[df_z['Recon'] == str(
                        recon)][str(i)], df_z[df_z['Recon'] == str(recon2)][str(i)], alternative='less')
                    # combine stats into dataframe
                    df_z_ttest_rel_results = pd.concat([df_z_ttest_rel_results, pd.DataFrame({'Recon 1': recon, 'Recon 2': recon2, 'T-statistic Paired T-test': ttest_rel_z_score[0],
                                                                                                'p-value Paired T-test': ttest_rel_z_score[1], 'Effect Size d': ttest_rel_z_score[2],
                                                                                                'Mean 1': ttest_rel_z_score[3], 'Mean 2': ttest_rel_z_score[4]}, index=[0])], axis=0)
                    df_z_shapiro_wilk_results = pd.concat([df_z_shapiro_wilk_results, pd.DataFrame({'Recon 1': recon, 'Recon 2': recon2, 'W-statistic Shapiro-Wilk test': shapiro_wilk_w,
                                                                                                    'p-value Shapiro-Wilk test': shapiro_wilk_p}, index=[0])], axis=0)
                    df_z_wilcoxon_results = pd.concat([df_z_wilcoxon_results, pd.DataFrame({'Recon 1': recon, 'Recon 2': recon2, 'Test statistic Wilcoxon Signed Rank Test': wilcoxon_results[0],
                                                                                            'p-value Wilcoxon Signed Rank Test': wilcoxon_results[1], 'Effect Size r': wilcoxon_results[2],
                                                                                            'Median 1': wilcoxon_results[3], 'Median 2': wilcoxon_results[4]}, index=[0])], axis=0)
        # print('T-test rel z-score results')
        # print(df_z_ttest_rel_results)
        # print('Shapiro-Wilk z-score results')
        # print(df_z_shapiro_wilk_results)
        # print('Wilcoxon z-score results')
        # print(df_z_wilcoxon_results)

        # combine df_z_ttest_rel_results, df_z_shapiro_wilk_results, df_z_wilcoxon_results into one dataframe "stats_all"
        df_z_ttest_rel_results = df_z_ttest_rel_results.reset_index(
            drop=True)
        df_z_shapiro_wilk_results = df_z_shapiro_wilk_results.reset_index(
            drop=True)
        df_z_wilcoxon_results = df_z_wilcoxon_results.reset_index(drop=True)
        stats_all = pd.concat(
            [df_z_ttest_rel_results, df_z_shapiro_wilk_results, df_z_wilcoxon_results], axis=1)
        # check if the p-value for the Shapiro-Wilk test is less than 0.05,
        # if so, check if the p-value for the Wilcoxon test is less than 0.005,
        # if so, then add a tally to the "significant_result" column. 
        # If the p-value for the Shapiro-Wilk test is greater than 0.05 and 
        # the p-value of the paired ttest is less than 0.005, then add a tally to the "significant_result" column.
        stats_all['significant_result'] = 0
        alpha_shapiro_wilk = 0.05
        alpha_wilcoxon = 0.05 / 9
        alpha_paired_ttest = 0.05 / 9
        for index, row in stats_all.iterrows():
            if float(row['p-value Shapiro-Wilk test']) < 0.05:
                if float(row['p-value Wilcoxon Signed Rank Test']) < alpha_wilcoxon:
                    stats_all.at[index, 'significant_result'] = 1
            elif float(row['p-value Shapiro-Wilk test']) > 0.05:
                if float(row['p-value Paired T-test']) < alpha_paired_ttest:
                    stats_all.at[index, 'significant_result'] = 1
        # print(stats_all)
        stats_all.to_csv(main_path+'/stats/'+file.stem + '_walk_length_' +
                         str(i)+'_stats_all.csv', index=False)
        print(str(stats_all['significant_result'].sum())+' significant results out of '+str(len(stats_all['significant_result'])/2)+' total results')



        # save dataframe to csv
        df_z_ttest_rel_results.to_csv(
            main_path+'/stats/'+file.stem+'_walk_length_'+str(i)+'_z_score_ttest_rel_results.csv', index=False)
        df_z_shapiro_wilk_results.to_csv(
            main_path+'/stats/'+file.stem+'_walk_length_'+str(i)+'_shapiro_wilk_results.csv', index=False)
        df_z_wilcoxon_results.to_csv(
            main_path+'/stats/'+file.stem+'_walk_length_'+str(i)+'_wilcoxon_results.csv', index=False)

2 significant results out of 3.0 total results
1 significant results out of 3.0 total results
1 significant results out of 3.0 total results
0 significant results out of 3.0 total results
0 significant results out of 3.0 total results
2 significant results out of 3.0 total results
3 significant results out of 3.0 total results
3 significant results out of 3.0 total results
2 significant results out of 3.0 total results
2 significant results out of 3.0 total results
2 significant results out of 3.0 total results
2 significant results out of 3.0 total results
2 significant results out of 3.0 total results
2 significant results out of 3.0 total results
2 significant results out of 3.0 total results
3 significant results out of 3.0 total results
3 significant results out of 3.0 total results
3 significant results out of 3.0 total results
2 significant results out of 3.0 total results
3 significant results out of 3.0 total results
2 significant results out of 3.0 total results
2 significant

Make Box Plots

In [157]:
# Read data
main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/'

file_list = Path(main_path).glob('*batch??.csv')
for file in file_list:
    print(file)
    df = pd.read_csv(file)
    print(file.stem)
    # melt data for boxplot
    dd=pd.melt(df,id_vars=['Recon'],value_vars=['1','2','3','4','5'],var_name='Walk Length')
    # seaborn boxplot with hue based on recon method
    sns.boxplot(x='Walk Length',y='value',data=dd,hue='Recon')
    # plt.show()
    plt.ylabel('Pearson Score')
    # save figure
    plt.savefig(main_path+'/plots/'+file.stem+'_box_plot.png')
    plt.close()

/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/volume_weighted_all_percent_batch16.csv
volume_weighted_all_percent_batch16
/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/count_all_percent_batch11.csv
count_all_percent_batch11
/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/count_all_percent_batch12.csv
count_all_percent_batch12
/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/volume_weighted_all_percent_batch13.csv
volume_weighted_all_percent_batch13
/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/volume_weighted_all_percent_batch17.csv
volume_weighted_all_percent_batch17
/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/count_all_percent_batch15.csv
count_all_percent_batch15
/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/count_all_percent_batch18.csv
count_all_percent_batch18
/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/volume_weighted_all_percent_batch15.csv
volume_weighted_all_percent_batch15
/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/count_all_per

Summarizing Number of Significant Results

In [215]:
# read in all stats_all files from walk length 4
main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/stats'
file_list = Path(main_path).glob('*walk_length_4_stats_all.csv')
df_list_count = []
df_list_vol = []
# df_list_mean_length = []
for file in file_list:
    if 'count' in file.stem:
        df_list_count.append(pd.read_csv(file))
    elif 'vol' in file.stem:
        df_list_vol.append(pd.read_csv(file))
    # elif 'mean_length' in file.stem:
    #     df_list_mean_length.append(pd.read_csv(file))
df_count = pd.concat(df_list_count)
df_vol = pd.concat(df_list_vol)
# df_mean_length = pd.concat(df_list_mean_length)

# define a function to calculate the Fraction of significant results for each Recon 1 vs Recon 2 method comparison
# where the value of 'Recon 1' contains recon1 and of 'Recon 2' contains recon2
def sig_frac(df, recon1, recon2):
    df = df[(df['Recon 1'].str.contains(recon1)) & (df['Recon 2'].str.contains(recon2))]
    sig_frac = df['significant_result'].sum() / (len(df['significant_result']))
    return sig_frac

# Calculate the Fraction of significant results for each Recon 1 vs Recon 2 method comparison
# for count where the value of 'Recon 1' contains DTI and of 'Recon 2' contains MSMT
sig_frac_count_dm = sig_frac(df_count, 'DTI', 'MSMT')
print("Fraction of significant paired differences Streamline Count: DTI vs MSMT")
print(sig_frac_count_dm)
# for vol where the value of 'Recon 1' contains DTI and of 'Recon 2' contains MSMT
sig_frac_vol_dm = sig_frac(df_vol, 'DTI', 'MSMT')
print("Fraction of significant paired differences Volume Weighted Streamline Count: DTI vs MSMT")
print(sig_frac_vol_dm)
# for mean length where the value of 'Recon 1' contains DTI and of 'Recon 2' contains MSMT
# sig_frac_mean_length_dm = sig_frac(df_mean_length, 'DTI', 'MSMT')
# print("Fraction of paired significant differences Mean Length: DTI vs MSMT")
# print(sig_frac_mean_length_dm)
sig_frac_count_gm = sig_frac(df_count, 'GQI', 'MSMT')
print("Fraction of significant paired differences Streamline Count: GQI vs MSMT")
print(sig_frac_count_gm)
sig_frac_vol_gm = sig_frac(df_vol, 'GQI', 'MSMT')
print("Fraction of significant paired differences Volume Weighted Streamline Count: GQI vs MSMT")
print(sig_frac_vol_gm)
# sig_frac_mean_length_gm = sig_frac(df_mean_length, 'GQI', 'MSMT')
# print("Fraction of paired significant differences Mean Length: GQI vs MSMT")
# print(sig_frac_mean_length_gm)
sig_frac_count_dg = sig_frac(df_count, 'DTI', 'GQI')
print("Fraction of significant paired differences Streamline Count: DTI vs GQI")
print(sig_frac_count_dg)
sig_frac_vol_dg = sig_frac(df_vol, 'DTI', 'GQI')
print("Fraction of significant paired differences Volume Weighted Streamline Count: DTI vs GQI")
print(sig_frac_vol_dg)
# sig_frac_mean_length_dg = sig_frac(df_mean_length, 'DTI', 'GQI')
# print("Fraction of paired significant differences Mean Length: DTI vs GQI")
# print(sig_frac_mean_length_dg)


Fraction of significant paired differences Streamline Count: DTI vs MSMT
0.35
Fraction of significant paired differences Volume Weighted Streamline Count: DTI vs MSMT
0.9
Fraction of significant paired differences Streamline Count: GQI vs MSMT
0.35
Fraction of significant paired differences Volume Weighted Streamline Count: GQI vs MSMT
0.95
Fraction of significant paired differences Streamline Count: DTI vs GQI
0.15
Fraction of significant paired differences Volume Weighted Streamline Count: DTI vs GQI
0.35


In [207]:
# read in z-score paired t-test results from all batches in main_path/stats with walk length 4
verbose = False
main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/'
# main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v3/scrambled_dataset/'
file_list = Path(main_path+'/stats').glob('volume_*_walk_length_4_z_score_ttest_rel_results.csv')
df_ii = pd.DataFrame()
for file in file_list:
    if verbose:
        print(file)
    df_ii = pd.concat([df,pd.read_csv(file)],axis=0)

df_dm = get_df_int(df=df_ii, recon1='DTI Node Volume Weighted Streamline Count', recon2='MSMT CSD SIFT2 Node Volume Weighted Streamline Count', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_dm)
# save out mean, std, max, min to one csv
df_dm.mean().to_csv(main_path+'/stats/walk_4_DTI_msmt_volume_weighted_paired_ttest_results_mean.csv')
df_dm.std().to_csv(main_path+'/stats/walk_4_DTI_msmt_volume_weighted_paired_ttest_results_std.csv')
df_dm.max().to_csv(main_path+'/stats/walk_4_DTI_msmt_volume_weighted_paired_ttest_results_max.csv')
df_dm.min().to_csv(main_path+'/stats/walk_4_DTI_msmt_volume_weighted_paired_ttest_results_min.csv')

df_gm = get_df_int(df=df_ii, recon1='GQI Node Volume Weighted Streamline Count', recon2='MSMT CSD SIFT2 Node Volume Weighted Streamline Count', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_gm)
# save out mean, std, max, min to one csv
df_gm.mean().to_csv(main_path+'/stats/walk_4_GQI_msmt_volume_weighted_paired_ttest_results_mean.csv')
df_gm.std().to_csv(main_path+'/stats/walk_4_GQI_msmt_volume_weighted_paired_ttest_results_std.csv')
df_gm.max().to_csv(main_path+'/stats/walk_4_GQI_msmt_volume_weighted_paired_ttest_results_max.csv')
df_gm.min().to_csv(main_path+'/stats/walk_4_GQI_msmt_volume_weighted_paired_ttest_results_min.csv')

df_dq = get_df_int(df=df_ii, recon1='DTI Node Volume Weighted Streamline Count', recon2='GQI Node Volume Weighted Streamline Count', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_dq)
# save out mean, std, max, min to one csv
df_dq.mean().to_csv(main_path+'/stats/walk_4_DTI_GQI_volume_weighted_paired_ttest_results_mean.csv')
df_dq.std().to_csv(main_path+'/stats/walk_4_DTI_GQI_volume_weighted_paired_ttest_results_std.csv')
df_dq.max().to_csv(main_path+'/stats/walk_4_DTI_GQI_volume_weighted_paired_ttest_results_max.csv')
df_dq.min().to_csv(main_path+'/stats/walk_4_DTI_GQI_volume_weighted_paired_ttest_results_min.csv')

# read in z-score paired t-test results from all batches in main_path/stats with walk length 4
verbose = False
df_vol_ttest = pd.DataFrame()
df_vol_ttest = pd.concat([df_dm.mean(),df_dm.std(),df_gm.mean(),df_gm.std(),df_dq.mean(),df_dq.std()],axis=1)
df_vol_ttest.columns = ['DTI_MSMT_mean','DTI_MSMT_std','GQI_MSMT_mean','GQI_MSMT_std','DTI_GQI_mean','DTI_GQI_std']
df_vol_ttest.to_csv(main_path+'/stats/walk_4_volume_weighted_paired_ttest_results.csv')

/tmp/ipykernel_619693/2977612436.py:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_dm.mean().to_csv(main_path+'/stats/walk_4_DTI_msmt_volume_weighted_paired_ttest_results_mean.csv')
/tmp/ipykernel_619693/2977612436.py:18: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_dm.std().to_csv(main_path+'/stats/walk_4_DTI_msmt_volume_weighted_paired_ttest_results_std.csv')
/tmp/ipykernel_619693/2977612436.py:27: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_gm.mean().to_csv(main_path+

In [184]:
# read in z-score wilcoxon signed rank test results from all batches in main_path/stats with walk length 4
verbose = False
main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/'
# main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v3/scrambled_dataset/'
file_list = Path(main_path+'/stats').glob('volume_*_walk_length_4_wilcoxon_results.csv')
df_ii = pd.DataFrame()
for file in file_list:
    if verbose:
        print(file)
    df_ii = pd.concat([df,pd.read_csv(file)],axis=0)

df_dm = get_df_int(df=df_ii, recon1='DTI Node Volume Weighted Streamline Count', recon2='MSMT CSD SIFT2 Node Volume Weighted Streamline Count', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_dm)
# save out mean, std, max, min to one csv
df_dm.mean().to_csv(main_path+'/stats/walk_4_DTI_msmt_volume_weighted_wilcoxon_results_mean.csv')
df_dm.std().to_csv(main_path+'/stats/walk_4_DTI_msmt_volume_weighted_wilcoxon_results_std.csv')
df_dm.max().to_csv(main_path+'/stats/walk_4_DTI_msmt_volume_weighted_wilcoxon_results_max.csv')
df_dm.min().to_csv(main_path+'/stats/walk_4_DTI_msmt_volume_weighted_wilcoxon_results_min.csv')

df_gm = get_df_int(df=df_ii, recon1='GQI Node Volume Weighted Streamline Count', recon2='MSMT CSD SIFT2 Node Volume Weighted Streamline Count', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_gm)
# save out mean, std, max, min to one csv
df_gm.mean().to_csv(main_path+'/stats/walk_4_GQI_msmt_volume_weighted_wilcoxon_results_mean.csv')
df_gm.std().to_csv(main_path+'/stats/walk_4_GQI_msmt_volume_weighted_wilcoxon_results_std.csv')
df_gm.max().to_csv(main_path+'/stats/walk_4_GQI_msmt_volume_weighted_wilcoxon_results_max.csv')
df_gm.min().to_csv(main_path+'/stats/walk_4_GQI_msmt_volume_weighted_wilcoxon_results_min.csv')

df_dq = get_df_int(df=df_ii, recon1='DTI Node Volume Weighted Streamline Count', recon2='GQI Node Volume Weighted Streamline Count', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_dq)
# save out mean, std, max, min to one csv
df_dq.mean().to_csv(main_path+'/stats/walk_4_DTI_GQI_volume_weighted_wilcoxon_results_mean.csv')
df_dq.std().to_csv(main_path+'/stats/walk_4_DTI_GQI_volume_weighted_wilcoxon_results_std.csv')
df_dq.max().to_csv(main_path+'/stats/walk_4_DTI_GQI_volume_weighted_wilcoxon_results_max.csv')
df_dq.min().to_csv(main_path+'/stats/walk_4_DTI_GQI_volume_weighted_wilcoxon_results_min.csv')

# combine all volume weighted wilcoxon signed rank test results at walk length 4
verbose = False
df_vol_wil = pd.DataFrame()
df_vol_wil = pd.concat([df_dm.mean(),df_dm.std(),df_gm.mean(),df_gm.std(),df_dq.mean(),df_dq.std()],axis=1)
df_vol_wil.columns = ['DTI_MSMT_mean','DTI_MSMT_std','GQI_MSMT_mean','GQI_MSMT_std','DTI_GQI_mean','DTI_GQI_std']
df_vol_wil.to_csv(main_path+'/stats/walk_4_volume_weighted_wilcoxon_results.csv')

/tmp/ipykernel_619693/127232398.py:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_dm.mean().to_csv(main_path+'/stats/walk_4_DTI_msmt_volume_weighted_wilcoxon_results_mean.csv')
/tmp/ipykernel_619693/127232398.py:18: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_dm.std().to_csv(main_path+'/stats/walk_4_DTI_msmt_volume_weighted_wilcoxon_results_std.csv')
/tmp/ipykernel_619693/127232398.py:27: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_gm.mean().to_csv(main_path+'/stats/wal

In [185]:
# read in z-score paired t-test results from all batches in main_path/stats with walk length 4
verbose = False
main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/'
# main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v3/scrambled_dataset/'
file_list = Path(main_path+'/stats').glob('count_*_walk_length_4_z_score_ttest_rel_results.csv')
df_ii = pd.DataFrame()
for file in file_list:
    if verbose:
        print(file)
    df_ii = pd.concat([df,pd.read_csv(file)],axis=0)

df_dm = get_df_int(df=df_ii, recon1='DTI Streamline Count', recon2='MSMT CSD SIFT2 Streamline Count', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_dm)
# save out mean, std, max, min to one csv
df_dm.mean().to_csv(main_path+'/stats/walk_4_DTI_msmt_count_paired_ttest_results_mean.csv')
df_dm.std().to_csv(main_path+'/stats/walk_4_DTI_msmt_count_paired_ttest_results_std.csv')
df_dm.max().to_csv(main_path+'/stats/walk_4_DTI_msmt_count_paired_ttest_results_max.csv')
df_dm.min().to_csv(main_path+'/stats/walk_4_DTI_msmt_count_paired_ttest_results_min.csv')

df_gm = get_df_int(df=df_ii, recon1='GQI Streamline Count', recon2='MSMT CSD SIFT2 Streamline Count', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_gm)
# save out mean, std, max, min to one csv
df_gm.mean().to_csv(main_path+'/stats/walk_4_GQI_msmt_count_paired_ttest_results_mean.csv')
df_gm.std().to_csv(main_path+'/stats/walk_4_GQI_msmt_count_paired_ttest_results_std.csv')
df_gm.max().to_csv(main_path+'/stats/walk_4_GQI_msmt_count_paired_ttest_results_max.csv')
df_gm.min().to_csv(main_path+'/stats/walk_4_GQI_msmt_count_paired_ttest_results_min.csv')

df_dq = get_df_int(df=df_ii, recon1='DTI Streamline Count', recon2='GQI Streamline Count', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_dq)
# save out mean, std, max, min to one csv
df_dq.mean().to_csv(main_path+'/stats/walk_4_DTI_GQI_count_paired_ttest_results_mean.csv')
df_dq.std().to_csv(main_path+'/stats/walk_4_DTI_GQI_count_paired_ttest_results_std.csv')
df_dq.max().to_csv(main_path+'/stats/walk_4_DTI_GQI_count_paired_ttest_results_max.csv')
df_dq.min().to_csv(main_path+'/stats/walk_4_DTI_GQI_count_paired_ttest_results_min.csv')


# combine all count t-test results at walk length 4
verbose = False
df_count_ttest = pd.DataFrame()
df_count_ttest = pd.concat([df_dm.mean(),df_dm.std(),df_gm.mean(),df_gm.std(),df_dq.mean(),df_dq.std()],axis=1)
df_count_ttest.columns = ['DTI_MSMT_mean','DTI_MSMT_std','GQI_MSMT_mean','GQI_MSMT_std','DTI_GQI_mean','DTI_GQI_std']
df_count_ttest.to_csv(main_path+'/stats/walk_4_count_paired_ttest_results.csv')

/tmp/ipykernel_619693/4074295172.py:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_dm.mean().to_csv(main_path+'/stats/walk_4_DTI_msmt_count_paired_ttest_results_mean.csv')
/tmp/ipykernel_619693/4074295172.py:18: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_dm.std().to_csv(main_path+'/stats/walk_4_DTI_msmt_count_paired_ttest_results_std.csv')
/tmp/ipykernel_619693/4074295172.py:27: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_gm.mean().to_csv(main_path+'/stats/walk_4_GQI_m

In [186]:
# read in z-score wilcoxon signed rank test results from all batches in main_path/stats with walk length 4
verbose = False
main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/'
# main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v3/scrambled_dataset/'
file_list = Path(main_path+'/stats').glob('count_*_walk_length_4_wilcoxon_results.csv')
df_ii = pd.DataFrame()
for file in file_list:
    if verbose:
        print(file)
    df_ii = pd.concat([df,pd.read_csv(file)],axis=0)

df_dm = get_df_int(df=df_ii, recon1='DTI Streamline Count', recon2='MSMT CSD SIFT2 Streamline Count', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_dm)
# save out mean, std, max, min to one csv
df_dm.mean().to_csv(main_path+'/stats/walk_4_DTI_msmt_count_wilcoxon_results_mean.csv')
df_dm.std().to_csv(main_path+'/stats/walk_4_DTI_msmt_count_wilcoxon_results_std.csv')
df_dm.max().to_csv(main_path+'/stats/walk_4_DTI_msmt_count_wilcoxon_results_max.csv')
df_dm.min().to_csv(main_path+'/stats/walk_4_DTI_msmt_count_wilcoxon_results_min.csv')

df_gm = get_df_int(df=df_ii, recon1='GQI Streamline Count', recon2='MSMT CSD SIFT2 Streamline Count', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_gm)
# save out mean, std, max, min to one csv
df_gm.mean().to_csv(main_path+'/stats/walk_4_GQI_msmt_count_wilcoxon_results_mean.csv')
df_gm.std().to_csv(main_path+'/stats/walk_4_GQI_msmt_count_wilcoxon_results_std.csv')
df_gm.max().to_csv(main_path+'/stats/walk_4_GQI_msmt_count_wilcoxon_results_max.csv')
df_gm.min().to_csv(main_path+'/stats/walk_4_GQI_msmt_count_wilcoxon_results_min.csv')

df_dq = get_df_int(df=df_ii, recon1='DTI Streamline Count', recon2='GQI Streamline Count', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_dq)
# save out mean, std, max, min to one csv
df_dq.mean().to_csv(main_path+'/stats/walk_4_DTI_GQI_count_wilcoxon_results_mean.csv')
df_dq.std().to_csv(main_path+'/stats/walk_4_DTI_GQI_count_wilcoxon_results_std.csv')
df_dq.max().to_csv(main_path+'/stats/walk_4_DTI_GQI_count_wilcoxon_results_max.csv')
df_dq.min().to_csv(main_path+'/stats/walk_4_DTI_GQI_count_wilcoxon_results_min.csv')

# combine all volume weighted wilcoxon signed rank test results at walk length 4
verbose = False
df_count_wil = pd.DataFrame()
df_count_wil = pd.concat([df_dm.mean(),df_dm.std(),df_gm.mean(),df_gm.std(),df_dq.mean(),df_dq.std()],axis=1)
df_count_wil.columns = ['DTI_MSMT_mean','DTI_MSMT_std','GQI_MSMT_mean','GQI_MSMT_std','DTI_GQI_mean','DTI_GQI_std']
df_count_wil.to_csv(main_path+'/stats/walk_4_count_wilcoxon_results.csv')

/tmp/ipykernel_619693/1331690065.py:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_dm.mean().to_csv(main_path+'/stats/walk_4_DTI_msmt_count_wilcoxon_results_mean.csv')
/tmp/ipykernel_619693/1331690065.py:18: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_dm.std().to_csv(main_path+'/stats/walk_4_DTI_msmt_count_wilcoxon_results_std.csv')
/tmp/ipykernel_619693/1331690065.py:27: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_gm.mean().to_csv(main_path+'/stats/walk_4_GQI_msmt_coun

In [152]:
# read in shapiro-wilk results from all batches in main_path/stats with walk length 4
verbose = False
main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/stats/'

shapiro_file_list = Path(main_path).glob('volume_*_walk_length_4_shapiro_wilk_results.csv')
df_sh = pd.DataFrame()
for sh_file in shapiro_file_list:
    if verbose:
        print(sh_file)
    df_sh = pd.concat([df_sh,pd.read_csv(sh_file)],axis=0)
df_sh_dm = get_df_int(df=df_sh, recon1='DTI Node Volume Weighted Streamline Count', recon2='MSMT CSD SIFT2 Node Volume Weighted Streamline Count', i=None)
# get mean and standard deviation of W statistic and p-value
if verbose:
    print_stats(df=df_sh_dm)
# save out mean, std to one csv
df_sh_dm.mean().to_csv(main_path+'/walk_4_DTI_msmt_volume_weighted_shapiro_wilk_results_mean.csv')
df_sh_dm.std().to_csv(main_path+'/walk_4_DTI_msmt_volume_weighted_shapiro_wilk_results_std.csv')

df_sh_gm = get_df_int(df=df_sh, recon1='GQI Node Volume Weighted Streamline Count', recon2='MSMT CSD SIFT2 Node Volume Weighted Streamline Count', i=None)
# get mean and standard deviation of W statistic and p-value
if verbose:
    print_stats(df=df_sh_gm)
# save out mean, std to one csv
df_sh_gm.mean().to_csv(main_path+'/walk_4_GQI_msmt_volume_weighted_shapiro_wilk_results_mean.csv')
df_sh_gm.std().to_csv(main_path+'/walk_4_GQI_msmt_volume_weighted_shapiro_wilk_results_std.csv')

df_sh_dq = get_df_int(df=df_sh, recon1='DTI Node Volume Weighted Streamline Count', recon2='GQI Node Volume Weighted Streamline Count', i=None)
# get mean and standard deviation of W statistic and p-value
if verbose:
    print_stats(df=df_sh_dq)
# save out mean, std to one csv
df_sh_dq.mean().to_csv(main_path+'/walk_4_DTI_GQI_volume_weighted_shapiro_wilk_results_mean.csv')
df_sh_dq.std().to_csv(main_path+'/walk_4_DTI_GQI_volume_weighted_shapiro_wilk_results_std.csv')

# read in mean and std files for shapiro-wilk at walk 4, summarize results for different recon1 reecon2 combinations
verbose = False
shapiro_list = ['walk_4_DTI_msmt_volume_weighted_shapiro_wilk_results_mean.csv',
              'walk_4_DTI_msmt_volume_weighted_shapiro_wilk_results_std.csv',
              'walk_4_DTI_GQI_volume_weighted_shapiro_wilk_results_mean.csv',
              'walk_4_DTI_GQI_volume_weighted_shapiro_wilk_results_std.csv',
              'walk_4_GQI_msmt_volume_weighted_shapiro_wilk_results_mean.csv',
              'walk_4_GQI_msmt_volume_weighted_shapiro_wilk_results_std.csv']
df_summary = pd.DataFrame()
for s_file in shapiro_list:
    if verbose:
        print(s_file)
    df = pd.read_csv(main_path+'/'+s_file, index_col=0)
    df_summary = pd.concat([df_summary,df],axis=1)

df_summary.columns = ['DTI MSMT mean', 'DTI MSMT std', 'DTI GQI mean', 'DTI GQI std', 'GQI MSMT mean', 'GQI MSMT std']
df_summary.to_csv(main_path+'/walk_4_volume_weighted_shapiro_wilk_summary.csv')


shapiro_file_list = Path(main_path).glob('count_*_walk_length_4_shapiro_wilk_results.csv')
df_sh = pd.DataFrame()
for sh_file in shapiro_file_list:
    if verbose:
        print(sh_file)
    df_sh = pd.concat([df_sh,pd.read_csv(sh_file)],axis=0)
df_sh_dm = get_df_int(df=df_sh, recon1='DTI Streamline Count', recon2='MSMT CSD SIFT2 Streamline Count', i=None)
# get mean and standard deviation of W statistic and p-value
if verbose:
    print_stats(df=df_sh_dm)
# save out mean, std to one csv
df_sh_dm.mean().to_csv(main_path+'/walk_4_DTI_msmt_count_shapiro_wilk_results_mean.csv')
df_sh_dm.std().to_csv(main_path+'/walk_4_DTI_msmt_count_shapiro_wilk_results_std.csv')

df_sh_gm = get_df_int(df=df_sh, recon1='GQI Streamline Count', recon2='MSMT CSD SIFT2 Streamline Count', i=None)
# get mean and standard deviation of W statistic and p-value
if verbose:
    print_stats(df=df_sh_gm)
# save out mean, std to one csv
df_sh_gm.mean().to_csv(main_path+'/walk_4_GQI_msmt_count_shapiro_wilk_results_mean.csv')
df_sh_gm.std().to_csv(main_path+'/walk_4_GQI_msmt_count_shapiro_wilk_results_std.csv')

df_sh_dq = get_df_int(df=df_sh, recon1='DTI Streamline Count', recon2='GQI Streamline Count', i=None)
# get mean and standard deviation of W statistic and p-value
if verbose:
    print_stats(df=df_sh_dq)
# save out mean, std to one csv
df_sh_dq.mean().to_csv(main_path+'/walk_4_DTI_GQI_count_shapiro_wilk_results_mean.csv')
df_sh_dq.std().to_csv(main_path+'/walk_4_DTI_GQI_count_shapiro_wilk_results_std.csv')

# read in mean and std files for shapiro-wilk at walk 4, summarize results for different recon1 reecon2 combinations
verbose = False
shapiro_list = ['walk_4_DTI_msmt_count_shapiro_wilk_results_mean.csv',
              'walk_4_DTI_msmt_count_shapiro_wilk_results_std.csv',
              'walk_4_DTI_GQI_count_shapiro_wilk_results_mean.csv',
              'walk_4_DTI_GQI_count_shapiro_wilk_results_std.csv',
              'walk_4_GQI_msmt_count_shapiro_wilk_results_mean.csv',
              'walk_4_GQI_msmt_count_shapiro_wilk_results_std.csv']
df_summary = pd.DataFrame()
for s_file in shapiro_list:
    if verbose:
        print(s_file)
    df = pd.read_csv(main_path+'/'+s_file, index_col=0)
    df_summary = pd.concat([df_summary,df],axis=1)

df_summary.columns = ['DTI MSMT mean', 'DTI MSMT std', 'DTI GQI mean', 'DTI GQI std', 'GQI MSMT mean', 'GQI MSMT std']
df_summary.to_csv(main_path+'/walk_4_count_shapiro_wilk_summary.csv')




/tmp/ipykernel_619693/248381940.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sh_dm.mean().to_csv(main_path+'/walk_4_DTI_msmt_volume_weighted_shapiro_wilk_results_mean.csv')
/tmp/ipykernel_619693/248381940.py:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sh_dm.std().to_csv(main_path+'/walk_4_DTI_msmt_volume_weighted_shapiro_wilk_results_std.csv')
/tmp/ipykernel_619693/248381940.py:24: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sh_gm.mean().to_csv(main_path+'/walk

In [ ]:
# read in z-score paired t-test results from all batches in main_path/stats with walk length 4
main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/'
# main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v3/scrambled_dataset/'
file_list = Path(main_path+'/stats').glob('count_*_walk_length_4_z_score_ttest_rel_results.csv')
df = pd.DataFrame()
for file in file_list:
    print(file)
    df = pd.concat([df,pd.read_csv(file)],axis=0)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is GQI Streamline Count and Recon 2 is MSMT CSD SIFT2 Streamline Count
print('mean')
print(df[(df['Recon 1']=='GQI Streamline Count') & (df['Recon 2']=='MSMT CSD SIFT2 Streamline Count')].mean())
print('std')
print(df[(df['Recon 1']=='GQI Streamline Count') & (df['Recon 2']=='MSMT CSD SIFT2 Streamline Count')].std())
print('max')
print(df[(df['Recon 1']=='GQI Streamline Count') & (df['Recon 2']=='MSMT CSD SIFT2 Streamline Count')].max())
print('min')
print(df[(df['Recon 1']=='GQI Streamline Count') & (df['Recon 2']=='MSMT CSD SIFT2 Streamline Count')].min())

# save out mean, std, max, min to csv
df[(df['Recon 1']=='GQI Streamline Count') & (df['Recon 2']=='MSMT CSD SIFT2 Streamline Count')].mean().to_csv(main_path+'/stats/walk_4_gqi_msmt_count_paired_ttest_results_mean.csv')
df[(df['Recon 1']=='GQI Streamline Count') & (df['Recon 2']=='MSMT CSD SIFT2 Streamline Count')].std().to_csv(main_path+'/stats/walk_4_gqi_msmt_count_paired_ttest_results_std.csv')
df[(df['Recon 1']=='GQI Streamline Count') & (df['Recon 2']=='MSMT CSD SIFT2 Streamline Count')].max().to_csv(main_path+'/stats/walk_4_gqi_msmt_count_paired_ttest_results_max.csv')
df[(df['Recon 1']=='GQI Streamline Count') & (df['Recon 2']=='MSMT CSD SIFT2 Streamline Count')].min().to_csv(main_path+'/stats/walk_4_gqi_msmt_count_paired_ttest_results_min.csv')

In [ ]:
# read in paired t-test results from all batches in main_path/stats with walk length 4
main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v4/dataset/'
# main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v3/scrambled_dataset/'
file_list = Path(main_path+'/stats').glob('mean_length_*_walk_length_4_ttest_rel_results.csv')
df_ii = pd.DataFrame()
for file in file_list:
    if verbose:
        print(file)
    df_ii = pd.concat([df,pd.read_csv(file)],axis=0)
df_dm = get_df_int(df=df_ii, recon1='DTI Mean Length', recon2='MSMT CSD Mean Length', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_dm)
# save out mean, std, max, min to one csv
df_dm.mean().to_csv(main_path+'/stats/walk_4_DTI_msmt_mean_length_wilcoxon_results_mean.csv')
df_dm.std().to_csv(main_path+'/stats/walk_4_DTI_msmt_mean_length_wilcoxon_results_std.csv')
df_dm.max().to_csv(main_path+'/stats/walk_4_DTI_msmt_mean_length_wilcoxon_results_max.csv')
df_dm.min().to_csv(main_path+'/stats/walk_4_DTI_msmt_mean_length_wilcoxon_results_min.csv')

df_gm = get_df_int(df=df_ii, recon1='GQI Mean Length', recon2='MSMT CSD Mean Length', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_gm)
# save out mean, std, max, min to one csv
df_gm.mean().to_csv(main_path+'/stats/walk_4_GQI_msmt_mean_length_wilcoxon_results_mean.csv')
df_gm.std().to_csv(main_path+'/stats/walk_4_GQI_msmt_mean_length_wilcoxon_results_std.csv')
df_gm.max().to_csv(main_path+'/stats/walk_4_GQI_msmt_mean_length_wilcoxon_results_max.csv')
df_gm.min().to_csv(main_path+'/stats/walk_4_GQI_msmt_mean_length_wilcoxon_results_min.csv')

df_dq = get_df_int(df=df_ii, recon1='DTI Mean Length', recon2='GQI Mean Length', i=None)
# get mean and standard deviation, max, min of t-statistic and p-value for the row where Recon 1 is DTI Node Volume Weighted Streamline Count and Recon 2 is MSMT CSD SIFT2 Node Volume Weighted Streamline Count
if verbose:
    print_stats(df=df_dq)
# save out mean, std, max, min to one csv
df_dq.mean().to_csv(main_path+'/stats/walk_4_DTI_GQI_mean_length_wilcoxon_results_mean.csv')
df_dq.std().to_csv(main_path+'/stats/walk_4_DTI_GQI_mean_length_wilcoxon_results_std.csv')
df_dq.max().to_csv(main_path+'/stats/walk_4_DTI_GQI_mean_length_wilcoxon_results_max.csv')
df_dq.min().to_csv(main_path+'/stats/walk_4_DTI_GQI_mean_length_wilcoxon_results_min.csv')

# combine all volume weighted wilcoxon signed rank test results at walk length 4
verbose = False
df_ml_wil = pd.DataFrame()
df_ml_wil = pd.concat([df_dm.mean(),df_dm.std(),df_gm.mean(),df_gm.std(),df_dq.mean(),df_dq.std()],axis=1)
df_ml_wil.columns = ['DTI_MSMT_mean','DTI_MSMT_std','GQI_MSMT_mean','GQI_MSMT_std','DTI_GQI_mean','DTI_GQI_std']
df_ml_wil.to_csv(main_path+'/stats/walk_4_mean_length_wilcoxon_results.csv')

In [ ]:
shapiro_file_list = Path(main_path).glob('mean_path_length_*_walk_length_4_shapiro_wilk_results.csv')
df_sh = pd.DataFrame()
for sh_file in shapiro_file_list:
    if verbose:
        print(sh_file)
    df_sh = pd.concat([df_sh,pd.read_csv(sh_file)],axis=0)
df_sh_dm = get_df_int(df=df_sh, recon1='DTI Mean Length', recon2='MSMT CSD Mean Length', i=None)
# get mean and standard deviation of W statistic and p-value
if verbose:
    print_stats(df=df_sh_dm)
# save out mean, std to one csv
df_sh_dm.mean().to_csv(main_path+'/walk_4_DTI_msmt_mean_length_shapiro_wilk_results_mean.csv')
df_sh_dm.std().to_csv(main_path+'/walk_4_DTI_msmt_mean_length_shapiro_wilk_results_std.csv')

df_sh_gm = get_df_int(df=df_sh, recon1='GQI Mean Length', recon2='MSMT CSD Mean Length', i=None)
# get mean and standard deviation of W statistic and p-value
if verbose:
    print_stats(df=df_sh_gm)
# save out mean, std to one csv
df_sh_gm.mean().to_csv(main_path+'/walk_4_GQI_msmt_mean_length_shapiro_wilk_results_mean.csv')
df_sh_gm.std().to_csv(main_path+'/walk_4_GQI_msmt_mean_length_shapiro_wilk_results_std.csv')

df_sh_dq = get_df_int(df=df_sh, recon1='DTI Mean Length', recon2='GQI Mean Length', i=None)
# get mean and standard deviation of W statistic and p-value
if verbose:
    print_stats(df=df_sh_dq)
# save out mean, std to one csv
df_sh_dq.mean().to_csv(main_path+'/walk_4_DTI_GQI_mean_length_shapiro_wilk_results_mean.csv')
df_sh_dq.std().to_csv(main_path+'/walk_4_DTI_GQI_mean_length_shapiro_wilk_results_std.csv')

# read in mean and std files for shapiro-wilk at walk 4, summarize results for different recon1 reecon2 combinations
verbose = False
shapiro_list = ['walk_4_DTI_msmt_mean_length_shapiro_wilk_results_mean.csv',
              'walk_4_DTI_msmt_mean_length_shapiro_wilk_results_std.csv',
              'walk_4_DTI_GQI_mean_length_shapiro_wilk_results_mean.csv',
              'walk_4_DTI_GQI_mean_length_shapiro_wilk_results_std.csv',
              'walk_4_GQI_msmt_mean_length_shapiro_wilk_results_mean.csv',
              'walk_4_GQI_msmt_mean_length_shapiro_wilk_results_std.csv']
df_summary = pd.DataFrame()
for s_file in shapiro_list:
    if verbose:
        print(s_file)
    df = pd.read_csv(main_path+'/'+s_file, index_col=0)
    df_summary = pd.concat([df_summary,df],axis=1)

df_summary.columns = ['DTI MSMT mean', 'DTI MSMT std', 'DTI GQI mean', 'DTI GQI std', 'GQI MSMT mean', 'GQI MSMT std']
df_summary.to_csv(main_path+'/walk_4_mean_length_shapiro_wilk_summary.csv')
